In [ ]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.decomposition import PCA
import pandas as pd
import ast

In [ ]:
# import os
# directory = '/content/drive/My Drive/annotation/'

# images = []
# landmarks = []
# for filename in os.listdir(directory):
#   f = os.path.join(directory, filename)
#   if os.path.isfile(f):
#     with open(f) as file:
#       images.append(file.readline().split("\n")[0]+".jpg")
#       imagelabels = []
#       for line in file:  #for line_num,line in enumerate(file):
#         cooardinates = line.split(" , ")
#         cooardinates = (int(round(float(cooardinates[0]))), int(round(float(cooardinates[1]))))
#         imagelabels.append(cooardinates)
#       landmarks.append(imagelabels)

In [ ]:
# df = pd.DataFrame(data = {'images':images, 'landmarks': landmarks})
# df.to_csv('/content/drive/My Drive/helen.csv', index=False)

In [ ]:
col_names = []
for i in range(65):
  col_names.append(f'landmark_x_{int(i+1)}')
  col_names.append(f'landmark_y_{int(i+1)}')

In [ ]:
# df = pd.read_csv('/content/drive/My Drive/helen.csv')
# def array_to_columns(row):
#     new_cols = {}
#     landmarks = ast.literal_eval(row['landmarks'])
#     for i in range(0, len(landmarks), 3):
#       col_name = f'landmark_x_{int((i/3)+1)}'
#       new_cols[col_name] = landmarks[i][0]
#       col_name = f'landmark_y_{int((i/3)+1)}'
#       new_cols[col_name] = landmarks[i][1]
#     return pd.Series(new_cols)

# new_df = df.apply(array_to_columns, axis=1)
# new_df.columns = col_names
# new_df = pd.concat([df, new_df], axis=1)
# new_df.drop('landmarks', axis=1, inplace=True)
# new_df.to_csv('/content/drive/My Drive/helen_65.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/My Drive/helen_65.csv')

In [ ]:
df.head()

,images,landmark_x_1,landmark_y_1,landmark_x_2,landmark_y_2,landmark_x_3,landmark_y_3,landmark_x_4,landmark_y_4,landmark_x_5,...,landmark_x_61,landmark_y_61,landmark_x_62,landmark_y_62,landmark_x_63,landmark_y_63,landmark_x_64,landmark_y_64,landmark_x_65,landmark_y_65
0,2547802012_1.jpg,390,528,404,584,426,639,461,688,502,...,523,381,469,397,493,404,550,396,608,403
1,2547808775_1.jpg,307,367,308,405,317,443,336,476,366,...,357,257,331,267,332,279,359,270,389,274
2,2552816820_2.jpg,303,381,306,421,316,461,334,497,358,...,430,312,396,326,397,343,430,329,469,327
3,2550746669_1.jpg,556,779,569,849,587,920,600,990,637,...,613,565,587,604,596,633,625,594,671,590
4,2548725859_2.jpg,761,1097,782,1191,811,1286,853,1376,927,...,870,799,813,853,844,848,913,815,995,812


In [ ]:
# imgname = '2547802012_1.jpg'
# landmarks=df.loc[df['images'] == imgname].iloc[0][col_names]
# img = cv2.imread('/content/drive/My Drive/helen/'+imgname)
# for i in range(0,len(landmarks),2):
#   cv2.circle(img, (landmarks[i],landmarks[i+1]), 3, (0, 255, 0), -1)
# cv2_imshow(img)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()

In [ ]:
landmarks_dataset = np.array(df[col_names].values.tolist())
landmarks_dataset=landmarks_dataset.reshape(2330,65,2)
mean_landmark = landmarks_dataset.mean(axis=0)

In [ ]:
def procrustes(X, Y, scaling=True, reflection='best'):
    """
    A port of MATLAB's `procrustes` function to Numpy.

    Procrustes analysis determines a linear transformation (translation,
    reflection, orthogonal rotation and scaling) of the points in Y to best
    conform them to the points in matrix X, using the sum of squared errors
    as the goodness of fit criterion.

        d, Z, [tform] = procrustes(X, Y)

    Inputs:
    ------------
    X, Y    
        matrices of target and input coordinates. they must have equal
        numbers of  points (rows), but Y may have fewer dimensions
        (columns) than X.

    scaling 
        if False, the scaling component of the transformation is forced
        to 1

    reflection
        if 'best' (default), the transformation solution may or may not
        include a reflection component, depending on which fits the data
        best. setting reflection to True or False forces a solution with
        reflection or no reflection respectively.

    Outputs
    ------------
    d       
        the residual sum of squared errors, normalized according to a
        measure of the scale of X, ((X - X.mean(0))**2).sum()

    Z
        the matrix of transformed Y-values

    tform   
        a dict specifying the rotation, translation and scaling that
        maps X --> Y

    """

    n,m = X.shape
    ny,my = Y.shape

    muX = X.mean(0)
    muY = Y.mean(0)

    X0 = X - muX
    Y0 = Y - muY

    ssX = (X0**2.).sum()
    ssY = (Y0**2.).sum()

    # centred Frobenius norm
    normX = np.sqrt(ssX)
    normY = np.sqrt(ssY)

    # scale to equal (unit) norm
    X0 /= normX
    Y0 /= normY

    if my < m:
        Y0 = np.concatenate((Y0, np.zeros(n, m-my)),0)

    # optimum rotation matrix of Y
    A = np.dot(X0.T, Y0)
    U,s,Vt = np.linalg.svd(A,full_matrices=False)
    V = Vt.T
    T = np.dot(V, U.T)

    if reflection != 'best':

        # does the current solution use a reflection?
        have_reflection = np.linalg.det(T) < 0

        # if that's not what was specified, force another reflection
        if reflection != have_reflection:
            V[:,-1] *= -1
            s[-1] *= -1
            T = np.dot(V, U.T)

    traceTA = s.sum()

    if scaling:

        # optimum scaling of Y
        b = traceTA * normX / normY

        # standarised distance between X and b*Y*T + c
        d = 1 - traceTA**2

        # transformed coords
        Z = normX*traceTA*np.dot(Y0, T) + muX

    else:
        b = 1
        d = 1 + ssY/ssX - 2 * traceTA * normY / normX
        Z = normY*np.dot(Y0, T) + muX

    # transformation matrix
    if my < m:
        T = T[:my,:]
    c = muX - b*np.dot(muY, T)
    
    #transformation values 
    tform = {'rotation':T, 'scale':b, 'translation':c}
   
    return d, Z, tform

In [ ]:
#For each face in the dataset, align its landmarks with respect to the mean shape using Procrustes analysis
aligned_landmarks = np.zeros_like(landmarks_dataset)
transformations = []

for i in range(len(landmarks_dataset)):
    d,z,tfrom = procrustes(mean_landmark, landmarks_dataset[i])
    aligned_landmarks[i] = z
    transformations.append(tfrom)

In [ ]:
candidate_shapes = aligned_landmarks

In [ ]:
x_bar = candidate_shapes.mean(axis=0)

In [ ]:
rng = np.random.default_rng()
sampled_shapes =rng.choice(candidate_shapes, 15, replace=False)

In [ ]:
# img = cv2.imread('/content/drive/My Drive/helen/'+imgname)

# transform_matrix = np.zeros((3,3))
# transform_matrix[:2,:2] = transformations[0]['rotation'] * transformations[0]['scale']
# transform_matrix[:2,2] = transformations[0]['translation']
# transform_matrix[2,2] = 1.0

# # apply transformation to image
# rows, cols = img.shape[:2]
# transformed_image = cv2.warpAffine(img, transform_matrix[:2], (cols, rows))


# landmarkstest = aligned_landmarks[0]
# for i in range(0,65):
#   cv2.circle(transformed_image, (landmarkstest[i][0],landmarkstest[i][1]), 3, (0, 255, 0), -1)
# cv2_imshow(transformed_image)

In [ ]:
# # Loop through each row in the dataframe
# for index, row in df.iterrows():
#     # Get image name and landmarks
#     image_name = row['image_name']
#     landmarks = row['landmarks']
    
#     # Load image
#     img = cv2.imread(image_dir + image_name)
    
#     # Convert landmarks to numpy array
#     landmarks = np.array(landmarks)
    
#     # Extract facial landmarks
#     x1, y1 = np.min(landmarks, axis=0)
#     x2, y2 = np.max(landmarks, axis=0)
    
#     # Compute the center of the face
#     cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    
#     # Compute the width and height of the face
#     w, h = x2 - x1, y2 - y1
    
#     # Add some padding to the face region
#     padding = 0.3
#     x1 -= int(w * padding)
#     y1 -= int(h * padding)
#     x2 += int(w * padding)
#     y2 += int(h * padding)
    
#     # Crop the image around the face
#     face_img = img[y1:y2, x1:x2]
    
#     # Update landmarks to match the cropped image
#     landmarks[:, 0] -= x1
#     landmarks[:, 1] -= y1
    
#     # Save the cropped image
#     cv2.imwrite('cropped/' + image_name, face_img)
    
#     # Update the landmarks in the dataframe
#     df.at[index, 'landmarks'] = landmarks.tolist()